# DSCI 200 Case Study 1

Alex Zhang (3815 4290)

In [1]:
# Install if needed (Uncomment)
# install.packages('tidyverse')
# devtools::install_github("lbelzile/hecedsm")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done




── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpThBs0y/remotes21830d4f341/lbelzile-hecedsm-df0a544/DESCRIPTION’ ... OK
* preparing ‘hecedsm’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
  NB: this package now depends on R (>= 4.1.0)
  pipe |> or function shorthand \(...) syntax added in R 4.1.0.
  File(s) using such syntax:
    ‘SKD23_S2A.Rd’
* building ‘hecedsm_2025.06.18.tar.gz’



In [2]:
# Load Libraries
library(tidyverse)
library(hecedsm)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.6
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.1     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘hecedsm’


The following object is masked from ‘package:stringr’:

    words




# Milestone 1: Exploratory Data Analysis (EDA)

## Task 1: Choose your favourite dataset

In [13]:
# Item 1: Loading the dataset
virt_com <- BL22_E

### Item 2: Paper Summary

Many workers reported a preference for remote work or hybrid work as the COVID-19 pandemic was winding down. The authours, both from business backgrounds, aimed to answer how virtual communication affects the creative output and decision making abilities of groups working remotely. This is of particular interest to business management, as understanding how different collaborative environments affect group productivity are critical to making effective workplace policy decisions.

Their main hypothesis was that "virtual communication hampers idea generation," which was tested and supported through multiple avenues. Specifically relevant to this dataset, the study tracked the eye movement of participants to gauge how much time they spent looking around the room, at their partner, and at the task. The task also measured how many creative ideas were generated by each pair and how many of those were "high quality."

## Task 2: Explore data

In [19]:
head(virt_com)
dim(virt_com)

id,writer,cond,partner_time,task_time,room_time
<fct>,<lgl>,<fct>,<dbl>,<dbl>,<dbl>
1,TRUE,video,84.626,193.868,20.026
1,FALSE,video,122.026,144.806,33.150
2,TRUE,video,86.122,125.630,88.740
2,FALSE,video,89.828,116.518,93.500
3,FALSE,f2f,1.632,255.816,41.480
4,FALSE,video,118.694,160.956,17.578


[1] 270   6

- **Item 3**: There are 270 observations within the dataset, each corresponding to one of the 270 participants.
- **Item 4**: The `id` and `cond` variables are factors, writer is a `lgl` boolean. The remaining 3 time variables (`partner_time`, `task_time`, `room_time` are all numerical doubles.
- **Item 5**: No conversions are necessary. `id` and `cond` are already factors, not numerical or characters. `writer` is also already a logical true/false boolean, and the numerical columns are all correctly doubles.

In [79]:
# Check count of missing data
proportion_na <- virt_com |> summarize(
    id_na = sum(is.na(id)),
    writer_na = sum(is.na(writer)),
    cond_na = sum(is.na(cond)),
    partner_time_na = sum(is.na(partner_time)),
    task_time_na = sum(is.na(task_time)),
    room_time_na = sum(is.na(room_time)),
)
proportion_na

# Separate issue with the pairings
count(count(virt_com, id), id_group_size = n) # Demonstrate that there are some ids with only 1 entry
id_count_issue <- virt_com |>
    group_by(id) |>
    filter(n() == 1) |>
    ungroup(id) |>
    select(writer, cond)
count(id_count_issue, single_id_writer = writer)
count(id_count_issue, single_id_cond = cond)

id_na,writer_na,cond_na,partner_time_na,task_time_na,room_time_na
<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,0,0,0


id_group_size,n
<int>,<int>
1,22
2,124


single_id_writer,n
<lgl>,<int>
FALSE,7
TRUE,15


single_id_cond,n
<fct>,<int>
f2f,12
video,10


- **Item 6**: There are no missing data values for any variable in this dataset. However, there is a potential issue with missing group pairings. 22 IDs only belong to one observation. I am unable to find an explanation for this in the paper, as they seem to suggest that all 270 participants were paired. (126 in-person, 144 virtual) In this analysis, since I only am considering the raw data and not at the pairing, I will still consider these valid data points.

In [57]:
# Check for class imbalance
count(virt_com, writer)
count(virt_com, cond)

# Not relevant to this
# count(virt_com, id)
# count(virt_com, partner_time)
# count(virt_com, task_time)
# count(virt_com, room_time)

writer,n
<lgl>,<int>
FALSE,131
TRUE,139


cond,n
<fct>,<int>
f2f,126
video,144


- **Item 7**: There is no major class imbalance, class counts are within 15% of each other.